In [ ]:
%matplotlib widget

## Loading packages and functions

**Understand what the packages below are and do.**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 2-photon dataset
**Load the tiff stack corresponding to plane 65: *plane065.tif* and load it into a numpy array called *images*. A stack is a file that contains a set of images. It is therefore a three dimensional array (Z,X,Y) or (T,X,Y) depending on whether the different images are from different planes or the same plane at different times. You can also open this stack with ImageJ and browse through it.**

In [ ]:
from PIL import Image

img = Image.open(r"C:\Users\rportugues\zenith\2p\plane065.tif")
images = []
for i in range(img.n_frames):
    img.seek(i)
    images.append(np.array(img))


**Use np.shape to query the dimensions of the array *images*. Does this make sense from what you observed in ImageJ?**

In [ ]:
np.shape(images)

**Generate an anatomy of this plane by summing all the frames. It should look very similar to**

![](images/figure1.png)

**Do you know where you are in the fish brain?**

In [ ]:
original_anatomy=np.sum(images,0)
plt.figure()
plt.imshow(original_anatomy)

**Now we will use the function** phase_cross_correlation **from skimage. We want to perform sub-pixel corrections, say to 0.1 of a pixel, so make sure you use the** upsample_factor **argument. Save the shifts in the X and Y directions in two arrays called *Xs* and *Ys* and compute also a *total_motion* array (use pythagoras).**

In [ ]:
from skimage.registration import phase_cross_correlation
import math
frames=np.array(images)
Xs=np.zeros(np.shape(images)[0])
Ys=np.zeros(np.shape(images)[0])
total_motion=np.zeros(np.shape(images)[0])
for i in range(frames.shape[0]):
    X=phase_cross_correlation(original_anatomy, frames[i,:,:] ,upsample_factor=10, space='real')
    Xs[i]=X[0][0]
    Ys[i]=X[0][1]
    total_motion[i]=math.sqrt(Xs[i]*Xs[i]+Ys[i]*Ys[i])

**Plot the total motion of each frame. Do you think the motion arises from continuous drifts or acute motor events? In a second subplot, plot a scatter plot of the shift for each frame. You can choose to add some jitter so you can visualize dots that are superimposed on one another.**

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.plot(total_motion)
plt.subplot(1,2,2)
plt.scatter(Xs-0.01+0.02*np.random.rand(Xs.shape[0]),Ys-0.01+0.02*np.random.rand(Xs.shape[0]),s=0.5)
plt.show()

## Second order correction of motion artifact

**Correct the individual frames by performing an individual sub-pixel alignment frame by frame. In order to do this use the function** shift **from** scipy.image.

**Use this new corrected stack to generate a new aligned_anatomy. Display the difference image between the original and the aligned anatomies.**

In [ ]:
from scipy.ndimage import shift
aligned_frames=np.zeros(frames.shape)
for i in range(frames.shape[0]):
    aligned_frames[i,:,:]=shift(frames[i,:,:], (Xs[i],Ys[i]), output=None, order=3, mode='constant', cval=0.0, prefilter=True)
#   the commented lines below check that the shift was performed in the correct direction    
#   X=phase_cross_correlation(anatomy, frames[i,:,:] ,upsample_factor=10, space='real')
#   print(X)
#   Y=phase_cross_correlation(anatomy, aligned_frames[i,:,:] ,upsample_factor=10, space='real')
#   print(Y)


In [ ]:
aligned_anatomy=np.sum(aligned_frames,0)
plt.figure()
plt.imshow(original_anatomy-aligned_anatomy)

**Align the ORIGINAL frames to this new aligned_anatomy. Compute the total shift of each frame to this aligned_anatomy and plot this shift on top of the shift you obtained for the first alignment. Why are these different?**

In [ ]:
from skimage.registration import phase_cross_correlation
import math
Xs2=np.zeros(np.shape(images)[0])
Ys2=np.zeros(np.shape(images)[0])
total_motion2=np.zeros(np.shape(images)[0])
for i in range(frames.shape[0]):
    X=phase_cross_correlation(aligned_anatomy, frames[i,:,:] ,upsample_factor=10, space='real')
    Xs2[i]=X[0][0]
    Ys2[i]=X[0][1]
    total_motion2[i]=math.sqrt(Xs2[i]*Xs2[i]+Ys2[i]*Ys2[i])


In [ ]:
plt.figure()
plt.plot(total_motion)
plt.plot(total_motion2)
plt.show()

**Finally shift the original frames by this new amount to obtain a final stack. You can sum this final stack to obtain a final anatomy.**


In [ ]:
from scipy.ndimage import shift
final_frames=np.zeros(frames.shape)
for i in range(frames.shape[0]):
    final_frames[i,:,:]=shift(frames[i,:,:], (Xs2[i],Ys2[i]), output=None, order=3, mode='constant', cval=0.0, prefilter=True)
final_anatomy=np.sum(final_frames,0)


In [ ]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow(original_anatomy)
plt.subplot(1,3,2)
plt.imshow(aligned_anatomy)
plt.subplot(1,3,3)
plt.imshow(final_anatomy)

## Creating a correlation map

**Use the function** pearsonr **from** scipy.stats **to compute the correlation between the (summed) fluorescence time series of one pixel and that fluorescence time series of its eight surrounding pixels (ignore boundary pixels for now).**

**Plot it alongside the anatomy. It should look something like:**

![](images/figure2.png)

In [ ]:
from scipy.stats import pearsonr

correlation_map=np.zeros(final_anatomy.shape)
for i in range(final_anatomy.shape[0]):
    if i>0 and i<(final_anatomy.shape[0]-1):
        for j in range(final_anatomy.shape[1]):
            if j>0 and j<(final_anatomy.shape[1]-1):
                this_pixel=np.squeeze(final_frames[:,i,j])
                surr_pixels=np.squeeze(np.sum(np.sum(np.squeeze(final_frames[:,i-1:i+1,j-1:j+1]),2),1))-this_pixel
                C, _ = pearsonr(this_pixel, surr_pixels)
                correlation_map[i,j]=C
original_correlation_map=np.copy(correlation_map)   



In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(final_anatomy)
plt.subplot(1,2,2)
plt.imshow(correlation_map)
plt.show()

**Plot a histogram of the correlation values. The top left and the bottom left of the image are not within the fish brain. Plot a histogram of the correlation values in this part of the image to gain some insight into the distribution of correlation values of the noise.**

## ROI extraction

**We will now generate a function that extracts ROIs (regions of interest). The function should take 4 inputs: a correlation map, the final stack of aligned frames, a correlation threshold value and a maximum size of an ROI in pixels.**

**The function should:**
- Identify the pixel with the highest correlation value in the correlation map (the ROI seed). Initialize the list of pixels in this ROI with this seed.**
- Compute the individual correlations of the fluorescence time-series in this pixel with its 8 neighbours. A convenient way of idetifying the neighbours of a region is by dilating the reion by one piexl. You can do this by importing **morphology** from skimage and using **morphology.binary_dilation**.
- Add the neighbours whose correlation exceeds the threshold to the ROI.
- Repeat by looking at the neighbouring pixels of the new ROI and computing the correlation of their fluorescence with the total fluorescence of the ROI. Add the pixels to the ROI if they exceed the threshold. 
- Stop the function if no neighbouring pixels exceed the threshold or if the size of the ROI exceeds the maximum size (the last input to the function).
- The function should return the pixels within the ROI, the fluoresence trace of the ROI, the size of the ROI and the correlation map with the values of the pixels in the extracted ROI set to zero.

**Use the function you have written to extract 200 ROIs within the plane. Use a correlation threshold value of 0.8 and a maximum ROI size of 200 pixels.**

**Plot a figure with three subplots. The first one should be an image of the fluorescence of the 200 ROIs. Make sure that you plot the normalized or zscored fluorescence for each ROI to be able to compare them. Import** zscore **from** scipy.stats **to do this. The second one a map showing the ROIs extraces and the thirs one the original correlation map with the correlation of the extracted ROIs set to zero. It should look something like this:**

![](images/figure3.png)

**Save the array with the traces as a numpy data file.**